# Exploratory Data Analysis

I will do an EDA as continuation of the steelers EDA

First step: Import libraries and config

In [ ]:
# numerical libraries
import pandas as pd
import numpy as np
from datetime import datetime

# data handling libraries
import yaml

# visualization libraries
import seaborn as sns

# configs
pd.set_option('display.max_columns', None)

# paths
file_path = '../datasets/games.csv'
config_path = '../config/config.yaml'

second step: import data and general adjustments

In [ ]:
# import raw data
games_df_raw = pd.read_csv(file_path)

# we analyze only previous seasons
games_df = games_df_raw[games_df_raw['season'] < 2025].copy()

# season 1999 has no gametime, so we remove it
games_df = games_df[games_df['season'] > 1999].reset_index(drop=True)

# adjust datatype to date
games_df['gameday'] = pd.to_datetime(games_df['gameday'])

third step: generate df for each team and concatenate them

In [ ]:
# set of teams (ideally home and away teams are the same)
teams_set = set(games_df_raw['home_team']) and set(games_df_raw['away_team'])

# auxiliar for the concat
df_list = []

# add a designed df for each team
for team in teams_set:
    
    # select games played by the team
    team_df = games_df[(games_df['home_team'] == team)|(games_df['away_team'] == team)].copy().reset_index()

    # add a variable: focus team
    team_df['focus_team'] = team

    # add variable: status
    team_df['focus_team_status'] = np.where(team_df['home_team'] == team,'home','away')

    # add a variable: vs team
    team_df['versus_team'] = np.where(team_df['home_team'] == team, team_df['away_team'], team_df['home_team'])

    # add variables: winner
    team_df['focus_team_score'] = np.where(team_df['focus_team_status'] == 'home', team_df['home_score'], team_df['away_score'])
    team_df['versus_team_score'] = np.where(team_df['focus_team_status'] == 'away', team_df['home_score'], team_df['away_score'])
    team_df['winner'] = np.where(team_df['focus_team_score'] > team_df['versus_team_score'],1,0)

    # add variables: coaches
    team_df['focus_team_coach'] = np.where(team_df['focus_team_status'] == 'home', team_df['home_coach'], team_df['away_coach'])
    team_df['versus_team_coach'] = np.where(team_df['focus_team_status'] == 'away', team_df['home_coach'], team_df['away_coach'])

    # selection of columns
    team_df = team_df[['game_id', 'focus_team', 'focus_team_status', 'versus_team', 
                       'focus_team_score', 'versus_team_score', 'winner',
                       'season', 'game_type', 'gameday', 'gametime']]
    
    # add columns with info of last ten games
    for i in range(10,len(team_df)):
        team_df.loc[i,'focus_team_ltg_wins'] = sum(team_df.loc[range(i-10,i),'winner'])
        team_df.loc[i,'focus_team_ltg_score'] = sum(team_df.loc[range(i-10,i),'focus_team_score'])
    
    # append new df in the list
    team_df = df_list.append(team_df)

# concat all dataframes
focused_df = pd.concat(df_list).reset_index(drop=True)

# get versus team last ten games wins by merging df with itself
focused_df = pd.merge(
    focused_df,
    focused_df[['game_id','focus_team','focus_team_ltg_wins']],
    how = 'left',
    left_on = ['game_id', 'versus_team'],
    right_on = ['game_id','focus_team']
)

# get versus team last ten games score by merging df with itself
focused_df = pd.merge(
    focused_df,
    focused_df[['game_id','focus_team_x','focus_team_ltg_score']],
    how = 'left',
    left_on = ['game_id', 'versus_team'],
    right_on = ['game_id','focus_team_x']
)

# drop and rename columns generated by merge
focused_df = focused_df.drop(columns=['focus_team_y','focus_team_x_y'])
focused_df = focused_df.rename(columns = {
    'focus_team_x_x':'focus_team',
    'focus_team_ltg_wins_x':'focus_team_ltg_wins',
    'focus_team_ltg_score_x':'focus_team_ltg_score',
    'focus_team_ltg_wins_y':'versus_team_ltg_wins',
    'focus_team_ltg_score_y':'versus_team_ltg_score'
})

# drop games with null values
focused_df = focused_df.dropna(
    subset=['focus_team_ltg_wins','focus_team_ltg_score','versus_team_ltg_wins','versus_team_ltg_score']
).reset_index(drop=True)

# get config
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# select section of config
validation_config = config['data_preprocessing']

# categorize gametime into day periods
focused_df['day_period'] = pd.cut(
    focused_df['gametime'].str.slice(0,2).astype(int),
    bins=validation_config['hour_bins'],
    labels=validation_config['hour_labels']
)

step 4: some graphs

In [ ]:
sns.pairplot(
    focused_df,
    vars=['focus_team_ltg_wins','focus_team_ltg_score', 'versus_team_ltg_wins','versus_team_ltg_score']
)

In [ ]:
sns.histplot(
    data = focused_df['gametime'].str.replace(':','').astype(int),
    kde=True
)

step 5: final dataframe info

In [ ]:
focused_df.info()